In [ ]:
!pip install faker
!pip install confluent_kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 13.8 MB/s eta 0:00:00


In [ ]:
import time
from faker import Faker
from confluent_kafka import Producer
import json

fake = Faker()

bootstrap_servers = 'pkc-4r087.us-west2.gcp.confluent.cloud:9092'
sasl_plain_username = 'ZI565VPJU4RFGXOK'
sasl_plain_password = 'Vo96TdSbLJafQGbtMyii8IqAb7BPlQ3S++iR7awRblJAl1XyJWXbL+WGZcCdO8bA'
topic_name = 'TrafficUpdate'

# Create the Kafka producer
producer_config = {
    'bootstrap.servers': bootstrap_servers,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': sasl_plain_username,
    'sasl.password': sasl_plain_password,
}

producer = Producer(producer_config)


def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to topic {} [partition {}] at offset {}'.format(msg.topic(), msg.partition(), msg.offset()))


def get_registered_users():
    return {
        "name": fake.name(),
        "address": fake.address(),
        "message": fake.text(),
        "created_on": fake.year()
    }

while True:
    user = get_registered_users()

    # Convert user data to JSON format
    user_json = json.dumps(user)

    # Produce the message to Kafka
    producer.produce(topic_name, user_json, callback=delivery_report)
    producer.flush()  # Ensure message is sent

    time.sleep(30)

Message delivered to topic TrafficUpdate [partition 2] at offset 12
Message delivered to topic TrafficUpdate [partition 5] at offset 18
Message delivered to topic TrafficUpdate [partition 3] at offset 19
Message delivered to topic TrafficUpdate [partition 4] at offset 15
Message delivered to topic TrafficUpdate [partition 4] at offset 16
Message delivered to topic TrafficUpdate [partition 1] at offset 9
Message delivered to topic TrafficUpdate [partition 5] at offset 19
Message delivered to topic TrafficUpdate [partition 2] at offset 13
Message delivered to topic TrafficUpdate [partition 5] at offset 20
Message delivered to topic TrafficUpdate [partition 1] at offset 10
Message delivered to topic TrafficUpdate [partition 1] at offset 11
Message delivered to topic TrafficUpdate [partition 2] at offset 14
Message delivered to topic TrafficUpdate [partition 2] at offset 15
Message delivered to topic TrafficUpdate [partition 1] at offset 12
Message delivered to topic TrafficUpdate [partiti

In [ ]:
import csv
from confluent_kafka import Consumer, KafkaError
import json

bootstrap_servers = 'pkc-4r087.us-west2.gcp.confluent.cloud:9092'
sasl_plain_username = 'ZI565VPJU4RFGXOK'
sasl_plain_password = 'Vo96TdSbLJafQGbtMyii8IqAb7BPlQ3S++iR7awRblJAl1XyJWXbL+WGZcCdO8bA'
oup_id = 'my_consumer_group'
topic_name = 'TrafficUpdate'

csv_file_path = 'received_users.csv'

# Create the Kafka consumer
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': group_id,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': sasl_plain_username,
    'sasl.password': sasl_plain_password,
    'auto.offset.reset': 'earliest',
}

consumer = Consumer(consumer_config)
consumer.subscribe([topic_name])

try:
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)

        while True:
            msg = consumer.poll(1.0)

            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print('Error: {}'.format(msg.error()))
                    break

            try:
                user_json = json.loads(msg.value().decode('utf-8'))
                # Write the user data to the CSV file
                csv_writer.writerow([user_json['name'], user_json['address'], user_json['message'], user_json['created_on']])
                print("Received and recorded user:", user_json)
            except json.JSONDecodeError as e:
                print('Error decoding JSON: {}'.format(e))

except KeyboardInterrupt:
    pass
finally:
    consumer.close()
